In [57]:
import csv
import json
import requests
import os

In [58]:
endpoint = 'https://www.loc.gov/free-to-use'
parameters = {
    'fo' : 'json'
}

In [59]:
collection = 'libraries'

In [60]:
collection_list_response = requests.get(endpoint + '/' + collection, params=parameters)

In [61]:
collection_list_response.url

'https://www.loc.gov/free-to-use/libraries?fo=json'

In [62]:
collection_json = collection_list_response.json()

local_json_collection = '/Users/julia.cave.arbanas/si676-2024-data/collection_project/collection_list_response.json'

with open(local_json_collection, 'w', encoding='utf-8', newline='') as f:
    json.dump(collection_json, f, ensure_ascii=False, indent=4)
print(f'JSON data saved')

JSON data saved


In [63]:
collection_json.keys()

dict_keys(['breadcrumbs', 'content', 'content_is_post', 'description', 'disable_max_line_length', 'expert_resources', 'next', 'next_sibling', 'options', 'pages', 'portal', 'previous', 'previous_sibling', 'site_type', 'timestamp', 'title', 'type'])

In [64]:
for k in collection_json['content']['set']['items']:
    print(k)

{'image': '/static/portals/free-to-use/public-domain/libraries/libraries-1.jpg', 'link': '/resource/cph.3f05183/', 'title': 'For greater knowledge, on more subjects, use your library more often. Illinois WPA Arts Project, 1936-1941. Prints & Photographs Division'}
{'image': '/static/portals/free-to-use/public-domain/libraries/libraries-2.jpg', 'link': '/resource/highsm.20336/', 'title': 'Noyes Library for Young Children. Kensington, Maryland. Photo by Carol M. Highsmith,  2011. Prints & Photographs Division'}
{'image': '/static/portals/free-to-use/public-domain/libraries/libraries-3.jpg', 'link': '/resource/fsa.8d24709/', 'title': 'Bethune-Cookman College. Students in the library reading room, Daytona Beach, Florida. Gordon Parks, 1943. Prints & Photographs Division'}
{'image': '/static/portals/free-to-use/public-domain/libraries/libraries-4.jpg', 'link': '/resource/highsm.36052/', 'title': 'Public library in Antonito,  Colorado, near the New Mexico border. Photo by Carol M. Highsmith,

In [65]:
len(collection_json['content']['set']['items'])

62

In [66]:
collection_json['content']['set']['items'][0].keys()

dict_keys(['image', 'link', 'title'])

In [67]:
collection_set_list = '/Users/julia.cave.arbanas/si676-2024-data/collection_project/ftu-libraries-set-list.csv'
headers = ['image','link','title']

with open(collection_set_list, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for item in collection_json['content']['set']['items']:
        item['title'] = item['title'].rstrip()
        writer.writerow(item)
    print('wrote',collection_set_list)

wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project/ftu-libraries-set-list.csv
